In [2]:
import chromadb
from llama_index.core import VectorStoreIndex, StorageContext, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import JSONNodeParser

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

In [4]:
# === 0. 讀取原始資料 ===
# 確保路徑正確，指向你存放 JSON 的資料夾
documents = SimpleDirectoryReader(input_dir="../data/json").load_data()

In [5]:
# === 1. 初始化 JSON Parser ===
parser = JSONNodeParser()

In [30]:
# 1. 連接資料庫
db = chromadb.PersistentClient(path="./chroma_db")
collection_name = "my_json_collection"

# === 💀 毀滅步驟：刪除舊的重複資料 ===

In [31]:
# === 💀 毀滅步驟：刪除舊的重複資料 ===
try:
    db.delete_collection(collection_name)
    print(f"🗑️ 成功刪除舊的 Collection '{collection_name}'，準備重建...")
except Exception as e:
    print("查無此 Collection，可能是第一次執行，直接繼續。")

🗑️ 成功刪除舊的 Collection 'my_json_collection'，準備重建...


In [32]:
# === 2. 設定 ChromaDB 持久化層 ===
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection(
    "my_json_collection", 
    metadata={"hnsw:space": "cosine"}
)

### 特別補充：你用的 BGE-M3 很強
你正在使用的 Embedding 模型 BAAI/bge-m3 其實本身就是為了 混合檢索 設計的！

M3 的意思：Multi-Linguality (多語言), Multi-Granularity (多粒度), Multi-Functionality (多功能)。

它同時支援 Dense Vector (像 HNSW 用) 和 Sparse Vector (像 BM25 用)。

不過，要在 LlamaIndex 中完全發揮 BGE-M3 的「原生」混合能力比較進階（需要特定的 Retriever 設定）。

In [33]:
# === 3. 建立 Vector Store 和 Storage Context ===
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [34]:
# 這裡會自動：讀取 documents -> 用 parser 切分 -> 轉向量 -> 存入 chroma_db
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model,
    transformations=[parser] 
)

In [35]:
print(f"成功處理 {len(documents)} 份文件，索引已儲存至 chroma_db")

成功處理 1 份文件，索引已儲存至 chroma_db


In [10]:
# === 5. 測試查詢 ===
query_engine = index.as_query_engine()
response = query_engine.query("這份資料關於什麼？")
print(response)

ImportError: `llama-index-llms-openai` package not found, please run `pip install llama-index-llms-openai`

優化：
```
import chromadb
import os
from llama_index.core import VectorStoreIndex, StorageContext, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import JSONNodeParser

# === 1. 設定環境與參數 ===
# 確保這些變數跟之前一樣
input_dir = "./data/json"
persist_dir = "./chroma_db"
collection_name = "my_json_collection"

# 初始化 Parser
parser = JSONNodeParser()

# === 2. 連接 ChromaDB ===
db = chromadb.PersistentClient(path=persist_dir)

# 取得 Collection
chroma_collection = db.get_or_create_collection(
    collection_name,
    metadata={"hnsw:space": "cosine"}
)

# 設定 Storage Context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# === 3. 核心邏輯：判斷要「載入」還是「新建」 ===
# 檢查目前 Collection 裡面有幾筆資料
doc_count = chroma_collection.count()

if doc_count > 0:
    print(f"✅ 偵測到資料庫已有 {doc_count} 筆資料，直接載入索引（不消耗 Token/時間）...")
    
    # 直接從 Vector Store 載入索引
    # 注意：這裡不需要傳入 documents，也不需要 parser
    index = VectorStoreIndex.from_vector_store(
        vector_store,
        embed_model=embed_model,
    )
    
else:
    print("⚠️ 資料庫為空，開始讀取檔案並計算 Embedding (這會花一點時間)...")
    
    # 1. 讀取檔案
    documents = SimpleDirectoryReader(input_dir=input_dir).load_data()
    
    # 2. 建立新索引並存入
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context, # 指向我們的 Chroma
        embed_model=embed_model,
        transformations=[parser]         # 切分 JSON
    )
    print("🎉 索引建立完成並已儲存！")

# === 4. 測試查詢 ===
query_engine = index.as_query_engine()
response = query_engine.query("這份資料關於什麼？")
print("-" * 20)
print(response)
```

## Retriver

In [16]:
retriever = index.as_retriever()
result = retriever.retrieve("nvidia 股價創新高影響？")

In [ ]:
result

In [13]:
result[0].metadata

{'file_path': '/Users/dingtseng/Documents/python_rag_class/demo_1/llamaIndex-tutorial/chroma-tutorial/../data/json/techorange-2024-10-16.json',
 'file_name': 'techorange-2024-10-16.json',
 'file_type': 'application/json',
 'file_size': 40751,
 'creation_date': '2025-12-18',
 'last_modified_date': '2025-01-03'}

In [11]:
print(result[0])

Node ID: 1727c12f-8cf5-48e3-ad65-d93abe360a0e
Text: title NVIDIA 股價創新高 逼近取代 Apple 成為全球最具價值公司 content NVIDIA (NVDA.O)
的股價在週一以歷史最高價收盤，將這家 AI 芯片巨頭推向有望取代 Apple (AAPL.O)
成為全球最具價值公司的邊緣。隨著投資者對於其當前及下一代 AI 處理器的需求充滿信心，這家位於加州聖塔克拉拉的公司股價上漲了 2.4%，收於
138.07 美元。 早在六月，NVIDIA
一度成為全球最具價值的公司，但隨後被微軟超越。近幾個月來，這三大科技巨頭的市值一直相差無幾。最新的漲幅使 NVIDIA 的市值達到 3.39
兆美元，略低於 Apple 的 3.52 兆美元，且高於微軟的 3.12 兆美元。 在 Alphabet
(GOOGL.O)、微軟、亞馬遜...
Score:  0.737



## TIPS: metadata 有 filters 可以去過濾

In [36]:
from llama_index.core.response.notebook_utils import display_source_node
retriever = index.as_retriever(similarity_top_k=3)
retrieved_nodes = retriever.retrieve("股價創新高影響？")

for node in retrieved_nodes:
    display_source_node(node, source_length=100)

**Node ID:** e4673b4c-1829-4e27-8476-eb8fd482cf17<br>**Similarity:** 0.6511675945653475<br>**Text:** title NVIDIA 股價創新高 逼近取代 Apple 成為全球最具價值公司
content NVIDIA (NVDA.O) 的股價在週一以歷史最高價收盤，將這家 AI 芯片巨頭推向有望取代...<br>

**Node ID:** b8a977b8-f370-46d4-932d-d9a335719a0d<br>**Similarity:** 0.5830028371838138<br>**Text:** title Netflix 用戶增長放緩，廣告收入成未來關鍵
content Netflix 預計在本週四公布其六個季度以來最慢的用戶增長，這主要歸因於打擊共享密碼措施的效益逐漸減弱。分析師估計...<br>

**Node ID:** e603a11c-5ad5-49c7-8318-7e8421340e62<br>**Similarity:** 0.5808336162390303<br>**Text:** title 改善工業能源效率從何做起？報告揭 3 大行動每年全球一起省下 2 兆美元
content PwC 發布《2024 淨零排放經濟指數報告》，指出全球減碳速度停滯，與前一年度相較，202...<br>

In [37]:
# 檢查目前索引中的總節點數
print(f"目前索引中的總節點數: {len(index.docstore.docs)}")

目前索引中的總節點數: 0


In [38]:
# 直接問 ChromaDB 裡面存了幾筆
print(f"資料庫真實節點數: {chroma_collection.count()}")

資料庫真實節點數: 10


In [39]:
print(f"資料庫中的 ID 列表: {chroma_collection.get()['ids']}")

資料庫中的 ID 列表: ['127c494a-0d89-45af-8d4b-ccc8cd1da5e9', 'e855b72a-cd97-4cf5-a3e6-d6ea6f613067', '9c7f32e2-df22-494f-935f-fa2a84571de0', '4a61673f-2203-4cd9-8ca5-69fd45e4da86', '99edec4b-0ccd-40d4-80bb-2659c25c572c', '027b5c5d-042f-4551-9d6a-d44739ab8386', 'e4673b4c-1829-4e27-8476-eb8fd482cf17', 'b8a977b8-f370-46d4-932d-d9a335719a0d', '3c1e7ff4-3f18-424e-bc65-6f875473d00f', 'e603a11c-5ad5-49c7-8318-7e8421340e62']


In [40]:
data = chroma_collection.get()

print(f"總共有 {len(data['ids'])} 筆資料\n")

for i, text in enumerate(data['documents'][:20]):
    print(f"--- Node {i+1} ---")
    # 只印出前 50 個字避免洗版
    print(text[:100]) 
    print("")

總共有 10 筆資料

--- Node 1 ---
title 【台灣陷入尷尬處境】半導體耗電量狂飆又得淨零轉型，核電會是最佳解？
content 台灣企業滿足了全球高達 68% 的晶片製造供應，隨之而來的是大量能源需求——根據綠色和平組織的預測，20

--- Node 2 ---
title Google 簽下協議買核電，建 7 座小型反應爐為 AI 與環保需求
content Google 宣布與能源新創公司 Kairos Power 簽下協議，將於未來興建 7 座小型模組化

--- Node 3 ---
title 蘋果策略大轉彎？傳放棄每年更新產品，VR 挫敗考慮推智慧眼鏡
content 根據蘋果記者 Mark Gurman 近來的爆料，他指出蘋果正在考慮放棄每年更新旗下產品，不再強制遵守原先的升

--- Node 4 ---
title 【碳費上路】全國徵收對象達 500 家！政府打算怎麼使用收到的 60 億？
content 環境部長彭啟明 14 日到立法院社福環衛委員會專題報告「台灣的碳費收費標準決議」，會中有多位立委

--- Node 5 ---
title 【10/15 快訊 Top 5】台積電稱「沒有新計畫」否認歐洲擴廠，將專注當前全球佈局
content 今天，《TechOrange》同樣精選出 5 則國內外科技新聞大事，讓你掌握科技動態

--- Node 6 ---
title 【直攻排隊結帳痛點】「AI 計時器」讓店家月營收增 4%！背後用了什麼魔法？
content 撰文‧張如嫻
飛捷已是全球領先的 POS 機廠，卻不甘只做硬體，近年來積極發展軟體事業，將 A

--- Node 7 ---
title NVIDIA 股價創新高 逼近取代 Apple 成為全球最具價值公司
content NVIDIA (NVDA.O) 的股價在週一以歷史最高價收盤，將這家 AI 芯片巨頭推向有望取代 Ap

--- Node 8 ---
title Netflix 用戶增長放緩，廣告收入成未來關鍵
content Netflix 預計在本週四公布其六個季度以來最慢的用戶增長，這主要歸因於打擊共享密碼措施的效益逐漸減弱。分析師估計，Ne

--- Node 9 ---
title 打擊詐騙、降低人力成本都能靠 AI，專家指金融業將被 AI 重

### 坑：如果重複存了，ｋ就會失效，因為retriver會自動過濾分數完全一模一樣的文章

## query
如果用 index.as_query_engine 就不用自己手動 retrive

In [41]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="gemma3:12b", request_timeout=600.0)

In [43]:
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("nvidia 股價創新高影響？")
print(response) # ???
print(response.source_nodes) # 查看檢索時用了哪些節點

NVIDIA 股價創新高將標準普爾 500 指數推向歷史新高，且 NVIDIA、Apple 和微軟合計約占標準普爾 500 指數的五分之一，對指數的日常漲跌具有重大影響。
[NodeWithScore(node=TextNode(id_='e4673b4c-1829-4e27-8476-eb8fd482cf17', embedding=None, metadata={'file_path': '/Users/dingtseng/Documents/python_rag_class/demo_1/llamaIndex-tutorial/chroma-tutorial/../data/json/techorange-2024-10-16.json', 'file_name': 'techorange-2024-10-16.json', 'file_type': 'application/json', 'file_size': 40751, 'creation_date': '2025-12-18', 'last_modified_date': '2025-01-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c622f256-d402-4a67-8eb5-4bda6ebdfa34', node_type='4', metadata={'file_path': '/Users/dingtseng/Documents/python_rag_class/demo_1/llamaIndex-tutorial/chroma-tutorial/../data/jso

In [ ]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response, show_source=True, source_length=100)

Final Response: NVIDIA 股價創新高將標準普爾 500 指數推向歷史新高，且 NVIDIA、Apple
和微軟合計約占標準普爾 500 指數的五分之一，對指數的日常漲跌具有重大影響。
______________________________________________________________________
Source Node 1/2
Node ID: e4673b4c-1829-4e27-8476-eb8fd482cf17
Similarity: 0.7373100070099398
Text: title NVIDIA 股價創新高 逼近取代 Apple 成為全球最具價值公司 content NVIDIA (NVDA.O)
的股價在週一以歷史最高價收盤，將這家 AI 芯片巨頭推向有望取代...
______________________________________________________________________
Source Node 2/2
Node ID: b8a977b8-f370-46d4-932d-d9a335719a0d
Similarity: 0.6028750778552528
Text: title Netflix 用戶增長放緩，廣告收入成未來關鍵 content Netflix
預計在本週四公布其六個季度以來最慢的用戶增長，這主要歸因於打擊共享密碼措施的效益逐漸減弱。分析師估計...


: 